# Assignment 2: 3D compressor stage design

### Design the NASA rotor 67 compressor stage using the "free vortex design"

The NASA rotor 67 is a well-known validation test case for turbomachinery CFD codes. See for example: 
 * https://ntrs.nasa.gov/api/citations/20050196726/downloads/20050196726.pdf 
 * https://how4.cenaero.be/content/c2-nasa-rotor-67
 
The following data is given:
    
 * Mass flow rate: $\dot m = 35$ 
 * Rotational speed: rpm = 16000
 * Rotor blade tip speed: $U_{tip} = 430$ m/s
 * **Total to total** pressure ratio: 1.6
 * Hub to tip ratio: $r_{hub}/r_{tip} = 0.375$
 * Static inlet conditions: 
     * Pressure $P = 0.8$ bar
     * Temperature $T = 3$ °C

For the working fluid, use the assumption of a calorically perfect gas (constant specific heats) for air with $R_{gas} = 287$ J/kg/K and $\gamma=1.4$. 

Furthermore, for the calculations below, assume:
 * a purely axial inflow velocity at the stage
 * a constant axial velocity across the whole stage
 * a purely axial velocity at the outlet of the stage
 * no losses. 

### Tasks: 
 
First, design the compressor stage at mid-span and calculate the inflow area, mid span radius, axial velocity, velocity angles, flow coefficient, loading factor, degree of reaction, etc. Note, the total to total pressure ratio is given, and the Euler equation relates the total enthaly change with the velocity triangles. 

Second, perform a 3D design of the compressor stage assuming a free vortex design. Show how the design parameters change along the span (radius of the stage). Also calculate the relative Mach number ($M=w/a$) at the inlet of the stage, $a$ is the speed of sound. 

Bonus: compare your design with the original NASA rotor 67 rotor. For example, use the first publication given above as a reference (compare the relative inlet Mach number or other quantities). 

Deliver a report of approximately 4 pages. Group of 2 students. **Do not share your results with the other groups.**


# The code below was introduced at the lecture and can be used for the assignment. Note, it is coded for a turbine stage.

In [1]:
def turbine_blades(phi, psi, R):
    
    c_x = 1.0
    U = c_x/phi
    
    A = np.array([[1, 1], [1, -1]])
    b = np.array([(1-2*R)/phi, (psi+1)/phi])
    x = np.linalg.solve(A, b)
    
    alfa2 = np.arctan(x[0])*(180/np.pi)
    beta2 = np.arctan(x[0] - U)*(180/np.pi)
    
    alfa3 = np.arctan(x[1] + U)*(180/np.pi)
    beta3 = np.arctan(x[1])*(180/np.pi)
    
    # Bezier 2nd order for stator and rotor blades
    t = np.linspace(0, 1, 21)
    F = np.column_stack(((1-t)**2, 2*t*(1-t),  t**2))
    
    # Profile thickness
    prof_th = 1.
    
    # Camber line stator
    stat = np.array([[U + x[1], 0.], [0, -1], [-x[0], -2.]])
    bez_stator = F.dot(stat)     
    
    # Stator profile pressure side (PS)
    statorPS = np.array([[U + x[1], 0.], [-prof_th/2, -1.], [-x[0], -2.]])
    bez_statorPS = F.dot(statorPS)
    
    # Stator profile suction side (SS)
    statorSS = np.array([[U + x[1], 0.], [prof_th/2., -1.], [-x[0], -2.]])
    bez_statorSS = F.dot(statorSS)
    
    stator = np.vstack((bez_statorSS, bez_statorPS[::-1]))
    
    # Camber line rotor
    rot = np.array([[0., -3.], [-x[0] + U , -4.], [-x[0] + U - x[1], -5.]])
    bez_stator = F.dot(rot)     
    
    # Rotor profile pressure side (PS)
    rotorPS = np.array([[0., -3.], [-x[0] + U - prof_th/2., -4.], [-x[0] + U - x[1], -5.]])
    bez_rotorPS = F.dot(rotorPS)
    
    # Rotor profile suction side (SS)
    rotorSS = np.array([[0., -3.], [-x[0] + U + prof_th/2., -4.], [-x[0] + U - x[1], -5.]])
    bez_rotorSS = F.dot(rotorSS)
    
    rotor = np.vstack((bez_rotorSS, bez_rotorPS[::-1]))
    
    return stator, rotor, x, U

In [2]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, interact_manual, interactive_output, Label
import numpy as np
from matplotlib import pyplot as plt

# Define initial parameters
init_phi = 0.4 # Flow coefficient
init_psi = 1 # Loading factor
init_R = 0.5 # Degree of reaction
init_pitch_stat = 3
init_pitch_rot = 3

# Define plot parameters
scale_fig1 = 1.5  
fontsize = 16

def turbine_gui(phi, psi, R, pitch_stat, pitch_rot, show_velocity=True): 
    
    
    offset_y = 0 if show_velocity == True else 0.25

    # Initialize figure 
    fig1, ax1 = plt.subplots(figsize=(scale_fig1*6.4, scale_fig1*4.8))

    # Set parameters left subplot
    ax1.set_xlabel('x', fontsize=fontsize)
    ax1.set_ylabel('y', fontsize=fontsize)
    ax1.set_xlim(-10, 10)
    ax1.set_ylim(-7, 2)
    
    [stator, rotor, x, U] = turbine_blades(phi, psi, R)
    
    for i in range(-10, 11):
        
        ax1.plot(stator[:,0] + i*pitch_stat, stator[:,1] + 0.25 - offset_y, lw=3, c='r')
        ax1.plot(rotor[:,0] + i*pitch_rot, rotor[:,1] - 0.25 + offset_y, lw=3, c='b')
    
    if show_velocity == True:
        
        offset = 0
        head_length=0.3
        head_width=0.3
        # Draw velocity profiles
        # Stator inlet
        ax1.arrow(offset, 1.5, offset-x[1]-U, -1, color='r', head_length=head_length, head_width=head_width, length_includes_head=True)  # C1

        # Stator outlet / rotor inlet
        ax1.arrow(offset, -2, -x[0], -1, color='r', head_length=head_length, head_width=head_width, length_includes_head=True, label='absolute velocity')  # C2
        ax1.arrow(offset, -2, -x[0]+U, -1, color='b', head_length=head_length, head_width=head_width, length_includes_head=True, label='relative velocity')  # W2
        ax1.arrow(offset-x[0]+U, -3, -U, 0, color='g', head_length=head_length, head_width=head_width, length_includes_head=True, label='rotational velocity')  # U

        # Rotor outlet
        ax1.arrow(offset, -5.5, -x[1]-U, -1, color='r', head_length=head_length, head_width=head_width, length_includes_head=True)  # C3
        ax1.arrow(offset, -5.5, -x[1], -1, color='b', head_length=head_length, head_width=head_width, length_includes_head=True)  # W3
        ax1.arrow(offset-x[1], -6.5, -U, 0, color='g', head_length=head_length, head_width=head_width, length_includes_head=True)  # U
        
        ax1.legend()
        
    # Update plot
    
    plt.show()

# Define interactive widgets
a = widgets.FloatSlider(value=init_phi, min=0, max=2, step=0.01, description=r'$\phi$')
b = widgets.FloatSlider(value=init_psi, min=0, max=5, step=0.01, description=r'$\psi$')
c = widgets.FloatSlider(value=init_R, min=0, max=2, step=0.01, description=r'$R$')
d = widgets.FloatSlider(value=init_pitch_stat, min=0, max=10, step=0.01, description='Pitch stator')
e = widgets.FloatSlider(value=init_pitch_rot, min=0, max=10, step=0.01, description='Pitch rotor')
f = widgets.Checkbox(True,  description='Show velocity triangles')

    
# Layout of widgets
ui0 = widgets.HBox([a, d])
ui1 = widgets.HBox([b, e])
ui2 = widgets.HBox([c, f])

ui = widgets.VBox([ui0, ui1, ui2])

# Activate interactivity with plot!
out = widgets.interactive_output(turbine_gui, {'phi': a, 'psi': b, 'R': c, 'pitch_stat': d, 'pitch_rot': e, 'show_velocity': f})
display(ui, out)


Output()